# 🎯 ConvNeXtPose Testing en Kaggle - Modelos L y M

Este notebook evalúa los modelos ConvNeXtPose L y M en Human3.6M Protocol 2.

**Datasets requeridos:**
- Human3.6M Dataset (con S9_ACT2_i6, S11_ACT2_i6, annotations)
- ConvNeXtPose Pre-trained Models (checkpoints .tar)

**GPU recomendada:** T4 x2 o P100

## 📦 PASO 1: Setup Inicial

In [ ]:
# Clonar repositorio
!git clone https://github.com/EstebanCabreraArbizu/ConvNeXtPose.git
%cd ConvNeXtPose

# Verificar versiones
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🗂️ PASO 2: Configurar Dataset Human3.6M

**IMPORTANTE:** Solo enlazamos el contenido del dataset dentro de `data/Human36M/`.
Los módulos Python originales (`dataset.py`, `Human36M.py`) permanecen intactos.

In [ ]:
# IMPORTANTE: Ajusta este path al nombre real de tu dataset en Kaggle
KAGGLE_DATASET_PATH = '/kaggle/input/human36m-dataset'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar que el dataset existe
import os
if not os.path.exists(KAGGLE_DATASET_PATH):
    print(f"❌ Dataset no encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/
    raise FileNotFoundError("Verifica el nombre del dataset y actualiza KAGGLE_DATASET_PATH")
else:
    print(f"✓ Dataset encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Contenido:")
    !ls {KAGGLE_DATASET_PATH}

### (Opcional) Diagnosticar Estructura del Dataset

Si tienes dudas sobre la estructura de tu dataset, ejecuta esta celda primero para ver cómo están organizadas las carpetas.

In [ ]:
# Diagnosticar estructura del dataset (útil para identificar carpetas anidadas)
!python diagnose_kaggle_dataset.py {KAGGLE_DATASET_PATH}

In [ ]:
# Ejecutar script de configuración
# IMPORTANTE: Esto enlaza el dataset DENTRO de data/Human36M/, NO reemplaza data/
# El script detecta automáticamente carpetas anidadas (ej: annotations (1)/annotations/)
!python setup_kaggle_dataset.py --kaggle-input {KAGGLE_DATASET_PATH} --project-root /kaggle/working/ConvNeXtPose

print("\n✅ Dataset enlazado en data/Human36M/")
print("✅ Módulos Python originales intactos en data/")
print("✅ Carpetas anidadas detectadas automáticamente")

In [ ]:
# Verificar que la estructura es correcta
!python setup_kaggle_dataset.py --verify /kaggle/working/ConvNeXtPose

## 🎯 PASO 3: Preparar Checkpoints

Extraer los modelos pre-entrenados.

In [ ]:
# IMPORTANTE: Ajusta según el nombre de tu dataset de modelos
MODELS_DATASET_PATH = '/kaggle/input/convnextpose-models'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar modelos disponibles
if os.path.exists(MODELS_DATASET_PATH):
    print("📦 Modelos disponibles:")
    !ls -lh {MODELS_DATASET_PATH}/models_tar/
else:
    print(f"❌ Dataset de modelos no encontrado: {MODELS_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/

In [ ]:
import os
import zipfile
import shutil

# Crear directorio para modelos
os.makedirs('output/model_dump', exist_ok=True)

# Función para extraer y organizar checkpoints
def extract_checkpoint(tar_path, model_name, expected_epoch=None):
    """Extrae checkpoint desde .tar y organiza la estructura correctamente
    
    Los .tar contienen una carpeta snapshot_XX.pth/ o archive/ con:
    - data.pkl
    - data/0, data/1, ... (archivos binarios del tensor)
    
    Esta función copia toda la estructura como snapshot_XX.pth/ que PyTorch puede cargar.
    """
    if not os.path.exists(tar_path):
        print(f"⚠️  Modelo {model_name} no encontrado en {tar_path}")
        return None
    
    print(f"📦 Extrayendo modelo {model_name} desde {tar_path}...")
    
    # Extraer a directorio temporal
    temp_dir = f'output/model_dump/temp_{model_name}'
    os.makedirs(temp_dir, exist_ok=True)
    
    with zipfile.ZipFile(tar_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
        files = zip_ref.namelist()
        print(f"   ✓ Extraído: {len(files)} archivos")
    
    # Buscar la carpeta del checkpoint (snapshot_XX.pth/ o archive/)
    found_checkpoints = []
    for item in os.listdir(temp_dir):
        item_path = os.path.join(temp_dir, item)
        if os.path.isdir(item_path):
            # Verificar que tenga data.pkl (indicador de checkpoint PyTorch)
            if os.path.exists(os.path.join(item_path, 'data.pkl')):
                found_checkpoints.append((item, item_path))
                print(f"   ✓ Checkpoint encontrado: {item}/")
    
    if not found_checkpoints:
        print(f"   ❌ No se encontró estructura de checkpoint válida")
        shutil.rmtree(temp_dir)
        return None
    
    # Procesar cada checkpoint encontrado
    epoch = None
    for ckpt_name, ckpt_path in found_checkpoints:
        # Determinar el nombre final
        import re
        match = re.search(r'snapshot_(\d+)', ckpt_name)
        if match:
            epoch = match.group(1)
            final_name = f'snapshot_{epoch}.pth.tar'
        else:
            # Si no tiene número, usar el epoch esperado
            if expected_epoch:
                epoch = str(expected_epoch)
                final_name = f'snapshot_{epoch}.pth.tar'
            else:
                final_name = f'{model_name}_checkpoint.pth.tar'
        
        dest_path = os.path.join('output/model_dump', final_name)
        
        # Copiar toda la carpeta del checkpoint
        if os.path.exists(dest_path):
            shutil.rmtree(dest_path)
        shutil.copytree(ckpt_path, dest_path)
        print(f"   ✓ Checkpoint copiado como: {final_name}")
        
        # Verificar estructura
        if os.path.exists(os.path.join(dest_path, 'data.pkl')):
            data_files = len(os.listdir(os.path.join(dest_path, 'data')))
            print(f"     - data.pkl: ✓")
            print(f"     - data/*: {data_files} archivos")
    
    # Limpiar temporal
    shutil.rmtree(temp_dir)
    print(f"   ✓ Limpieza temporal completada")
    
    return epoch

# Extraer modelo L (epoch 83 según el paper)
model_l_path = f'{MODELS_DATASET_PATH}/models_tar/ConvNeXtPose_L.tar'
epoch_l = extract_checkpoint(model_l_path, 'L', expected_epoch=83)

# Extraer modelo M (epoch 70 según el paper)
model_m_path = f'{MODELS_DATASET_PATH}/models_tar/ConvNeXtPose_M.tar'
epoch_m = extract_checkpoint(model_m_path, 'M', expected_epoch=70)

# Verificar checkpoints extraídos
print("\n📂 Checkpoints disponibles:")
!ls -lh output/model_dump/

# Mostrar información de epochs
if epoch_l:
    print(f"\n💡 Modelo L: Usa CHECKPOINT_EPOCH = {epoch_l}")
if epoch_m:
    print(f"💡 Modelo M: Usa CHECKPOINT_EPOCH = {epoch_m}")

print("\n⚠️  NOTA: Los checkpoints están en formato carpeta (snapshot_XX.pth.tar/)")
print("   PyTorch puede cargarlos usando torch.load('output/model_dump/snapshot_83.pth.tar')")

### ⚠️ Nota sobre Extracción de Checkpoints

Los archivos `.tar` de ConvNeXtPose son en realidad **archivos ZIP** con estructura anidada:
```
ConvNeXtPose_L.tar (archivo zip)
└── snapshot_83.pth/        ← Directorio
    ├── data.pkl
    ├── version
    └── data/               ← Carpeta con el checkpoint real
        └── 0, 1, 2...      ← Archivos binarios
```

La siguiente celda extrae y reorganiza correctamente el archivo `.pth` real.

In [ ]:
# Verificar checkpoints extraídos y detectar epoch
import glob
import re

print("🔍 Verificación de Checkpoints:\n")

# Buscar archivos .pth (no directorios)
checkpoints = [f for f in glob.glob('output/model_dump/snapshot_*.pth') 
               if os.path.isfile(f)]  # ← Solo archivos, no directorios

if checkpoints:
    print("✅ Checkpoints encontrados:\n")
    for ckpt in sorted(checkpoints):
        ckpt_name = os.path.basename(ckpt)
        size_mb = os.path.getsize(ckpt) / (1024 * 1024)
        
        # Extraer epoch
        match = re.search(r'snapshot_(\d+)', ckpt_name)
        if match:
            epoch = match.group(1)
            print(f"  ✓ {ckpt_name} ({size_mb:.1f} MB)")
            print(f"    → Usa CHECKPOINT_EPOCH = {epoch}\n")
else:
    print("❌ No se encontraron checkpoints .pth válidos\n")
    
    # Diagnosticar problema
    all_items = glob.glob('output/model_dump/snapshot_*')
    if all_items:
        print("⚠️  Se encontraron directorios en lugar de archivos:")
        for item in all_items:
            if os.path.isdir(item):
                print(f"    📁 {os.path.basename(item)} (directorio)")
                # Buscar dentro del directorio
                pth_inside = glob.glob(f"{item}/**/*.pth", recursive=True)
                if pth_inside:
                    print(f"       Contiene: {[os.path.basename(p) for p in pth_inside]}")
        
        print("\n💡 Solución: Re-ejecuta la celda anterior de extracción")
    else:
        print("💡 Solución: Verifica que MODELS_DATASET_PATH es correcto")

**⚠️ IMPORTANTE:** Ajusta el valor de `CHECKPOINT_EPOCH` en las siguientes celdas según el epoch de tu checkpoint extraído. En este caso detectamos `snapshot_83.pth`, así que usa `CHECKPOINT_EPOCH = 83`.

## 🚀 PASO 4: Ejecutar Testing

### Modelo L (Large)

In [ ]:
# Diagnóstico: Verificar que todos los componentes están listos
import sys
import os

print("🔍 Verificación de Componentes:\n")

# 1. Dataset - Verificar estructura DENTRO del proyecto
project_root = '/kaggle/working/ConvNeXtPose'
data_dir = os.path.join(project_root, 'data')
h36m_path = os.path.join(data_dir, 'Human36M')

print(f"1. Dataset (estructura del proyecto):")
print(f"   Project root: {project_root}")
print(f"   data/ exists: {os.path.exists(data_dir)}")
print(f"   data/dataset.py: {os.path.exists(os.path.join(data_dir, 'dataset.py'))}")
print(f"   data/Human36M/ exists: {os.path.exists(h36m_path)}")

if os.path.exists(h36m_path):
    print(f"   - Human36M.py: {os.path.exists(os.path.join(h36m_path, 'Human36M.py'))}")
    print(f"   - annotations: {os.path.exists(os.path.join(h36m_path, 'annotations'))}")
    print(f"   - images/S9: {os.path.exists(os.path.join(h36m_path, 'images', 'S9'))}")
    print(f"   - images/S11: {os.path.exists(os.path.join(h36m_path, 'images', 'S11'))}")

# 2. Checkpoints
checkpoint_dir = os.path.join(project_root, 'output/model_dump')
print(f"\n2. Checkpoints: {checkpoint_dir}")
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('snapshot_')]
    if checkpoints:
        print(f"   Disponibles: {', '.join(checkpoints)}")
        # Extraer epoch del checkpoint
        import re
        for ckpt in checkpoints:
            match = re.search(r'snapshot_(\d+)', ckpt)
            if match:
                epoch = match.group(1)
                print(f"   💡 Usa CHECKPOINT_EPOCH = {epoch} en la siguiente celda")
    else:
        print("   ⚠️  No se encontraron checkpoints")
else:
    print("   ❌ Directorio no existe")

# 3. Estructura del proyecto
print(f"\n3. Estructura del proyecto:")
critical_files = ['main/config.py', 'common/base.py', 'data/dataset.py', 'data/Human36M/Human36M.py']
for file_path in critical_files:
    full_path = os.path.join(project_root, file_path)
    exists = os.path.exists(full_path)
    status = "✓" if exists else "❌"
    print(f"   {status} {file_path}")

print("\n" + "="*60)
if all(os.path.exists(os.path.join(project_root, f)) for f in critical_files):
    print("✅ Todos los checks pasaron - Listo para testing")
else:
    print("❌ Algunos archivos faltan - Revisa la configuración")
print("="*60)

### Ejecutar Testing desde Python

**Estructura correcta:**
- ✅ Módulos Python originales en `data/` (dataset.py, Human36M.py)
- ✅ Dataset de Kaggle enlazado en `data/Human36M/` (images, annotations)
- ✅ `config.py` configura automáticamente los paths

In [ ]:
# Testing con estructura correcta del proyecto
import sys
import os

# 1. Cambiar al directorio main (donde está config.py)
os.chdir('/kaggle/working/ConvNeXtPose/main')

# 2. Importar config PRIMERO - esto configura automáticamente los paths
from config import cfg

# 3. Cargar variante ANTES de importar otros módulos
VARIANT = 'L'  # Cambiar a 'M' para modelo Medium
CHECKPOINT_EPOCH = 83  # ← AJUSTAR según tu checkpoint

print(f"\n{'='*60}")
print(f"  Testing ConvNeXtPose-{VARIANT}")
print(f"{'='*60}\n")

cfg.load_variant_config(VARIANT)
cfg.set_args('0')  # GPU 0

# 4. AHORA importar los demás módulos (config ya configuró sys.path)
import torch
import torch.backends.cudnn as cudnn
from base import Tester
import numpy as np
from tqdm import tqdm

cudnn.benchmark = True
cudnn.deterministic = False
cudnn.enabled = True

# 5. Crear tester y ejecutar
tester = Tester()
tester._make_batch_generator()
tester._make_model(CHECKPOINT_EPOCH)

print(f"\n🚀 Ejecutando testing en epoch {CHECKPOINT_EPOCH}...\n")

preds = []
with torch.no_grad():
    for itr, input_img in enumerate(tqdm(tester.batch_generator)):
        coord_out = tester.model(input_img)
        
        if cfg.flip_test:
            from utils.pose_utils import flip
            flipped_input_img = flip(input_img, dims=3)
            flipped_coord_out = tester.model(flipped_input_img)
            flipped_coord_out[:, :, 0] = cfg.output_shape[1] - flipped_coord_out[:, :, 0] - 1
            for pair in tester.flip_pairs:
                flipped_coord_out[:, pair[0], :], flipped_coord_out[:, pair[1], :] = \
                    flipped_coord_out[:, pair[1], :].clone(), flipped_coord_out[:, pair[0], :].clone()
            coord_out = (coord_out + flipped_coord_out)/2.
        
        coord_out = coord_out.cpu().numpy()
        preds.append(coord_out)

# Evaluar
preds = np.concatenate(preds, axis=0)
print(f"\n📊 Evaluando {len(preds)} predicciones...\n")
tester._evaluate(preds, cfg.result_dir)

print(f"\n✅ Testing completado!")
print(f"📂 Resultados guardados en: {cfg.result_dir}")

### Modelo M (Medium) - Opcional

In [ ]:
# Si tienes el checkpoint del modelo M, ejecuta esto:
# !python test.py --gpu 0 --epochs {CHECKPOINT_EPOCH} --variant M

## 📊 PASO 5: Verificar Resultados

In [ ]:
# Ver resultados generados
%cd ..
!ls -lh output/result/

# Leer log de resultados
import glob
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Últimas líneas del log ({os.path.basename(latest_log)}):")
    !tail -n 20 {latest_log}
else:
    print("⚠️  No se encontraron logs")

## 📈 PASO 6: Análisis de Resultados

In [ ]:
import re

def extract_mpjpe_from_log(log_path):
    """Extrae el MPJPE del log"""
    with open(log_path, 'r') as f:
        content = f.read()
    
    # Buscar patrón de MPJPE
    pattern = r'MPJPE.*?(\d+\.\d+)'
    matches = re.findall(pattern, content)
    
    if matches:
        return float(matches[-1])  # Último valor
    return None

# Extraer resultados
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    mpjpe = extract_mpjpe_from_log(latest_log)
    
    print("\n" + "="*60)
    print("  📊 RESULTADOS FINALES")
    print("="*60)
    
    if mpjpe:
        print(f"\n  MPJPE (Protocol 2): {mpjpe:.2f} mm")
        
        # Comparar con paper
        expected = {
            'L': 42.3,
            'M': 44.6
        }
        
        # Determinar variante
        for variant, expected_val in expected.items():
            diff = abs(mpjpe - expected_val)
            if diff < 5:
                print(f"\n  Variante detectada: {variant}")
                print(f"  Valor del paper: {expected_val:.2f} mm")
                print(f"  Diferencia: {mpjpe - expected_val:+.2f} mm")
                
                if diff < 2:
                    print("  ✅ Resultado excelente (dentro de ±2mm)")
                elif diff < 5:
                    print("  ✓ Resultado aceptable (dentro de ±5mm)")
                break
    else:
        print("\n⚠️  No se pudo extraer MPJPE del log")
        print("Revisa el log manualmente en output/log/")
    
    print("\n" + "="*60)
else:
    print("❌ No se encontraron logs de testing")

## 💾 PASO 7: Guardar Outputs

Kaggle guarda automáticamente todo en `/kaggle/working/`. Opcionalmente puedes copiar resultados específicos:

In [ ]:
# Crear resumen de resultados
import json
from datetime import datetime

summary = {
    'timestamp': datetime.now().isoformat(),
    'model': 'ConvNeXtPose-L',  # Cambiar según modelo testeado
    'checkpoint_epoch': CHECKPOINT_EPOCH,
    'dataset': 'Human3.6M Protocol 2',
    'mpjpe_mm': mpjpe if 'mpjpe' in locals() else None,
    'pytorch_version': torch.__version__,
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else None
}

# Guardar resumen
with open('output/result/test_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Resumen guardado en output/result/test_summary.json")
print("\n📄 Contenido:")
print(json.dumps(summary, indent=2))

## 🎉 Testing Completado!

Los resultados están en:
- **Logs**: `output/log/`
- **Resultados**: `output/result/`
- **Resumen JSON**: `output/result/test_summary.json`

Todos los archivos en `/kaggle/working/ConvNeXtPose/output/` se guardarán automáticamente cuando hagas commit del notebook.